# HelloWorld

In [6]:
sc.stop()


In [2]:
import sys
from random import random
from operator import add

print(sc.master)

NUM_SAMPLES = 2000000000

def inside(p):
    x, y = random(), random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

k8s://https://kubernetes:443
Pi is roughly 3.141648


# Spark SQL

In [4]:
import pyspark
sqlContext = pyspark.SQLContext(sc)

In [10]:
#! pip install pandas
data = [
    (1, 'a'), 
    (2, 'b'), 
    (3, 'c'), 
    (4, 'd'), 
    (5, 'e'), 
    (6, 'a'), 
    (7, 'b'), 
    (8, 'c'), 
    (9, 'd'), 
    (10, 'e')
]

# Convert a local data set into a DataFrame
df = spark.createDataFrame(data, ['numbers', 'letters'])

# Convert to a Pandas DataFrame for easy display
df.toPandas()

,numbers,letters
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,a
6,7,b
7,8,c
8,9,d
9,10,e


In [ ]:
# Register the DataFrame as a table
df.registerTempTable("mytable")

# Peform a simple select from the table
results = sqlContext.sql("select * from mytable")

# Convert the results to a Pandas DataFrame for easy viewing
results.toPandas()

In [ ]:
# Perform a query with a where clause and order by
results = sqlContext.sql("select * from mytable where numbers < 8 order by numbers desc")

# Convert the results to a Pandas DataFrame for easy viewing
results.toPandas()

In [ ]:
# Perform a more complex query on the table
results = sqlContext.sql("select letters, count(*) as count, avg(numbers) as avg, sum(numbers) as sum from mytable group by letters")

# Convert the results to a Pandas DataFrame for easy viewing
results.toPandas()

# Spark Dataframe

In [ ]:
# Count the number of rows in the DataFrame
print(df.count())
# View some rows
print(df.take(3))

In [ ]:
# Sort descending
descendingDf = df.orderBy(df.numbers.desc())

# View some rows
descendingDf.toPandas()

In [ ]:
# Filter the DataFrame
filtered = df.where(df.numbers < 5)

# Convert to Pandas DataFrame for easy viewing
filtered.toPandas()

In [ ]:
# import some more functions
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import avg
from pyspark.sql.functions import sum

# Perform aggregations on the DataFrame
agg = df.agg(
    avg(df.numbers).alias("avg_numbers"), 
    sum(df.numbers).alias("sum_numbers"),
    countDistinct(df.numbers).alias("distinct_numbers"), 
    countDistinct(df.letters).alias('distinct_letters')
)

# Convert the results to Pandas DataFrame
agg.toPandas()

In [ ]:
# View some summary statistics
df.describe().show()

# Spark RDD

In [ ]:
# Parallelize a data set converting from an Array to an RDD
rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
# Count the number of rows in the RDD
print(rdd.count())

In [ ]:
# View some rows
print(rdd.take(10))

In [ ]:
# Sort descending
descendingRdd = rdd.sortBy(lambda x: x, ascending = False)

# View some rows
print(descendingRdd.take(10))

In [ ]:
# Filter the RDD
filteredRdd = rdd.filter(lambda x: x < 5)

# View some rows
print(filteredRdd.take(10))

In [ ]:
# Map the RDD
rdd2 = rdd.map(lambda x: (x, x * 2))

# View some rows
print(rdd2.take(10))

In [ ]:
# Reduce the RDD by adding up all of the numbers
result = rdd.reduce(lambda a, b: a + b)

print(result)

In [ ]:
# Load a Text file from HDFS / MINIO
#textFile = sc.textFile("hdfs://...")

In [ ]:
# Save an RDD to HDFS
#textFile.saveAsTextFile("hdfs://...")

In [ ]:
# Parallelize a data set converting from an Array to an RDD
rdd = sc.parallelize(["aaa bbb ccc", "aaa bbb", "bbb ccc", "abc"])

# WordCount
results = rdd.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

# Get the Results
results.take(10)

In [3]:
sc

''

# Setup executors by hand

In [ ]:

sc.stop()
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("k8s://https://kubernetes:443")
         .setAppName("Notebook")
         .set("spark.executor.memory", "1g")
         .set("spark.executor.instances", "1")
         .set("spark.kubernetes.container.image", "cloudpg/spark-py:dodas-1.0.1")
         .set("spark.kubernetes.authenticate.driver.serviceAccountName","default")
         .set("spark.submit.deployMode", "client")
         #.set("spark.hadoop.fs.s3a.endpoint", "<minio host>:31311")
         #.set("spark.hadoop.fs.s3a.access.key", "admin")
         #.set("spark.hadoop.fs.s3a.secret.key", "adminminio")
         #.set("spark.hadoop.fs.s3a.fast.upload", "true")
         #.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         )

sc = SparkContext(conf = conf)


In [ ]:
#sql = SparkSession(sc)
#csv_df = sql.read.csv("s3a://test/file.csv")

# Cluster mode

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz

In [ ]:
!tar xzvf spark-2.4.3-bin-hadoop2.7.tgz

In [ ]:
# Stop pyspark shell executors to make room for new ones
#sc.stop()

#! /usr/local/spark/bin/spark-submit --master k8s://kubernetes:443 --deploy-mode cluster --name spark-pi2 \
#       --class org.apache.spark.examples.SparkPi --conf spark.executor.instances=2 \
#        --conf spark.kubernetes.container.image=dodasts/spark:v3.0.2 \
#            --conf spark.dynamicAllocation.maxExecutors=10 --conf spark.dynamicAllocation.enabled=true \
#            --conf spark.dynamicAllocation.shuffleTracking.enabled=true \
#            --conf spark.ui.prometheus.enabled=true \
#            --conf spark.executor.processTreeMetrics.enabled=true \
#            local:///usr/local/spark/examples/jars/spark-examples_2.12-3.0.1.jar 50000
                
! /usr/local/spark/bin/spark-submit --master k8s://kubernetes:443 --deploy-mode cluster --name spark-pi2 \
        --class org.apache.spark.examples.SparkPi -c spark.driver.memory=2g -c spark.executor.instances=30 \
        -c spark.ui.prometheus.enabled=true -c spark.kubernetes.driver.annotation.prometheus.io/scrape=true \
        -c spark.dynamicAllocation.enabled=true \
        -c spark.dynamicAllocation.executorIdleTimeout=5 -c spark.kubernetes.file.upload.path=/tmp/ \
        -c spark.dynamicAllocation.shuﬄeTracking.enabled=true -c spark.dynamicAllocation.maxExecutors=50 \
        -c spark.kubernetes.driver.annotation.prometheus.io/path=/metrics/executors/prometheus/ -c spark.kubernetes.driver.annotation.prometheus.io/port=4040 \
        -c spark.kubernetes.container.image=dodasts/spark:v3.0.2 \
        local:///usr/local/spark/examples/jars/spark-examples_2.12-3.0.1.jar 200000

#! /usr/local/spark/bin/spark-submit --master k8s://kubernetes:443 --deploy-mode cluster --name spark-pi3 \
#        -c spark.kubernetes.pyspark.pythonVersion=3 -c spark.dynamicAllocation.enabled=true \
#       -c spark.dynamicAllocation.executorIdleTimeout=5 -c spark.kubernetes.file.upload.path=/tmp/ \
#       -c spark.dynamicAllocation.shuﬄeTracking.enabled=true -c spark.dynamicAllocation.maxExecutors=50 \
#       -c spark.ui.prometheus.enabled=true c spark.kubernetes.driver.annotation.prometheus.io/scrape=true \
#       -c spark.kubernetes.driver.annotation.prometheus.io/path=/metrics/executors/prometheus/ -c spark.kubernetes.driver.annotation.prometheus.io/port=4040 \
#        -c spark.kubernetes.container.image=marcoverl/spark-py:v3.0.1 https://gist.githubusercontent.com/dongjoon-hyun/845d3063b3c79d7475cfc95e72d4ea08/raw/3b0baa240814e8ebb5060878328f3d3fe8cc229e/dynamic-pi.py 2000

20/10/11 19:42:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/10/11 19:42:23 INFO SparkKubernetesClientFactory: Auto-configuring K8S client using current context from users K8S config file
20/10/11 19:42:24 INFO KerberosConfDriverFeatureStep: You have not specified a krb5.conf file locally or via a ConfigMap. Make sure that you have the krb5.conf locally on the driver image.
20/10/11 19:42:35 INFO LoggingPodStatusWatcherImpl: State changed, new state: 
	 pod name: spark-pi2-d6c4fb75192f7c10-driver
	 namespace: default
	 labels: spark-app-selector -> spark-3a5d46fff2174e08a13a2f49e24fb6c4, spark-role -> driver
	 pod uid: bbf7ea16-a8c5-4641-a8bd-5de3bb837f1c
	 creation time: 2020-10-11T19:42:27Z
	 service account name: default
	 volumes: spark-local-dir-1, spark-conf-volume, default-token-bzd42
	 node name: N/A
	 start time: N

In [10]:
sc.stop()

In [1]:
#sc.stop()
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("k8s://https://kubernetes:443")
         .setAppName("Notebook")
# Prometheus monitoring        
#         .set("spark.jars.packages","io.prometheus.jmx:jmx_prometheus_javaagent:0.14.0")
         .set("spark.metrics.conf.*.sink.prometheusServlet.class","org.apache.spark.metrics.sink.PrometheusServlet")
         .set("spark.metrics.conf.*.sink.prometheusServlet.path","/metrics/prometheus")
         .set("spark.metrics.conf.master.sink.prometheusServlet.path","/metrics/master/prometheus")
         .set("spark.metrics.conf.applications.sink.prometheusServlet.path","/metrics/applications/prometheus")
         .set("master.source.jvm.class","org.apache.spark.metrics.source.JvmSource")
         .set("worker.source.jvm.class","org.apache.spark.metrics.source.JvmSource")
         .set("driver.source.jvm.class","org.apache.spark.metrics.source.JvmSource")
         .set("executor.source.jvm.class","org.apache.spark.metrics.source.JvmSource")
         .set("spark.ui.prometheus.enabled","true")
         .set("spark.executor.processTreeMetrics.enabled","true")
         .set("spark.kubernetes.driver.annotation.prometheus.io/scrape","true")
         .set("spark.kubernetes.driver.annotation.prometheus.io/port","4040")
         .set("spark.kubernetes.driver.annotation.prometheus.io/path", "/metrics/executors/prometheus/")
         .set("spark.kubernetes.driver.service.annotation.prometheus.io/scrape","true")
         .set("spark.kubernetes.driver.service.annotation.prometheus.io/port","4040")
         .set("spark.kubernetes.driver.service.annotation.prometheus.io/path", "/metrics/executors/prometheus/")
# end of monitoring
#         .set("spark.shuffle.service.enabled","true")
         .set("spark.dynamicAllocation.enabled","true")
#         .set("spark.dynamicAllocation.executorIdleTimeout","5")
         .set("spark.dynamicAllocation.shuﬄeTracking.enabled","true")
         .set("spark.dynamicAllocation.maxExecutors","50")
         .set("spark.executor.memory", "2g")
         .set("spark.executor.instances", "1")
         .set("spark.kubernetes.container.image", "marcoverl/spark-py:v3.0.1")
#         .set("spark.kubernetes.container.image", "dodasts/spark:v3.0.2")
         .set("spark.kubernetes.container.image.pullPolicy","Always")
         .set("spark.kubernetes.authenticate.driver.serviceAccountName","default")
#         .set("spark.jars.ivy", "/tmp")
#         .set("spark.jars","https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.872/aws-java-sdk-bundle-1.11.872.jar,https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.2.0/hadoop-aws-3.2.0.jar")
#         .set("spark.jars.packages","io.prometheus.jmx:jmx_prometheus_javaagent:0.14.0,org.apache.hadoop:hadoop-aws:3.2.0,com.amazonaws:aws-java-sdk-bundle:1.11.878")
         .set("spark.hadoop.fs.s3a.endpoint", "http://90.147.174.114:9000")
         .set("spark.hadoop.fs.s3a.path.style.access","true")
         .set("spark.hadoop.fs.s3a.access.key", "CCR")
         .set("spark.hadoop.fs.s3a.secret.key", "tutorialCCR")
         .set("spark.hadoop.fs.s3a.fast.upload", "true")
         .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .set("spark.hadoop.fs.s3a.committer.name", "directory")
         .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.internal.io.cloud.PathOutputCommitProtocol")
         .set("spark.sql.parquet.output.committer.class", "org.apache.spark.internal.io.cloud.BindingParquetOutputCommitter")
       )

sc = SparkContext(conf = conf)

In [10]:
file = sc.textFile("s3a://democcr/spark_notebook_error.txt")
file.count()

145

In [3]:
from pyspark.sql import SparkSession
sql = SparkSession(sc)
csv_df = sql.read.csv("s3a://verlato/dpc-covid19-ita-regioni-latest.csv")

In [6]:
csv_df.toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20
0,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
1,2020-07-03T17:00:00,ITA,13,Abruzzo,42.35122196,13.39843823,27,0,27,129,...,-4,6,2678,464,3104,194,3298,108363,71976,None
2,2020-07-03T17:00:00,ITA,17,Basilicata,40.63947052,15.80514834,0,0,0,4,...,2,2,373,27,401,3,404,40333,39553,NEL TOTALE CASI POSITIVI SONO COMPRESI ULTERIO...
3,2020-07-03T17:00:00,ITA,18,Calabria,38.90597598,16.59440194,4,0,4,21,...,0,1,1060,97,1159,23,1182,97852,95818,None
4,2020-07-03T17:00:00,ITA,15,Campania,40.83956555,14.25084984,35,0,35,162,...,7,10,4083,432,4682,30,4712,289253,142219,None
5,2020-07-03T17:00:00,ITA,08,Emilia-Romagna,44.49436681,11.341720800000001,87,9,96,915,...,8,27,23284,4267,27990,572,28562,508691,304202,None
6,2020-07-03T17:00:00,ITA,06,Friuli Venezia Giulia,45.6494354,13.76813649,6,0,6,62,...,6,4,2905,345,3315,3,3318,196975,108444,None
7,2020-07-03T17:00:00,ITA,12,Lazio,41.89277044,12.48366722,188,11,199,627,...,-5,11,6475,840,6008,2133,8141,346239,283710,None
8,2020-07-03T17:00:00,ITA,07,Liguria,44.41149315,8.9326992,40,3,43,240,...,2,3,8146,1558,8723,1264,9987,151324,80887,None
9,2020-07-03T17:00:00,ITA,03,Lombardia,45.46679409,9.190347404,241,41,282,9395,...,-150,115,67871,16675,88834,5389,94223,1064173,636860,None


In [6]:
csv_df.show()

+-------------------+-----+--------------+--------------------+-----------+------------------+--------------------+-----------------+--------------------+--------------------+---------------+--------------------+--------------+---------------+--------+--------------------+-----------------+-----------+-------+------------+--------------------+
|                _c0|  _c1|           _c2|                 _c3|        _c4|               _c5|                 _c6|              _c7|                 _c8|                 _c9|           _c10|                _c11|          _c12|           _c13|    _c14|                _c15|             _c16|       _c17|   _c18|        _c19|                _c20|
+-------------------+-----+--------------+--------------------+-----------+------------------+--------------------+-----------------+--------------------+--------------------+---------------+--------------------+--------------+---------------+--------+--------------------+-----------------+-----------+-----

In [11]:
sc.stop()


In [7]:
sc.getConf().getAll()

[('spark.dynamicAllocation.maxExecutors', '50'),
 ('spark.metrics.conf.*.sink.prometheusServlet.path', '/metrics/prometheus'),
 ('spark.kubernetes.driver.service.annotation.prometheus.io/port', '4040'),
 ('spark.kubernetes.driver.service.annotation.prometheus.io/scrape', 'true'),
 ('spark.hadoop.fs.s3a.path.style.access', 'true'),
 ('spark.metrics.conf.master.sink.prometheusServlet.path',
  '/metrics/master/prometheus'),
 ('spark.dynamicAllocation.executorIdleTimeout', '5'),
 ('spark.hadoop.fs.s3a.secret.key', 'tutorialCCR'),
 ('spark.app.name', 'Notebook'),
 ('spark.kubernetes.driver.annotation.prometheus.io/scrape', 'true'),
 ('spark.executor.processTreeMetrics.enabled', 'true'),
 ('spark.driver.extraClassPath',
  '/opt/conda/lib/python3.8/site-packages/sparkmonitor/listener.jar'),
 ('spark.executor.memory', '1g'),
 ('spark.executor.instances', '2'),
 ('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener'),
 ('spark.ui.prometheus.enabled', 'true'),
 ('spark.dyna

In [3]:
! pip install boto3

     |████████████████████████████████| 129 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 12 kB/s  eta 0:00:01     |████████████████████▊           | 4.3 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 4.7 MB/s  eta 0:00:01


In [4]:
import boto3

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
schema = ["employee_name", "department", "salary"]
  
  
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

print("approx_count_distinct: " + \
      str(df.select(approx_count_distinct("salary")).collect()[0][0]))

print("avg: " + str(df.select(avg("salary")).collect()[0][0]))

df.select(collect_list("salary")).show(truncate=False)

df.select(collect_set("salary")).show(truncate=False)

df2 = df.select(countDistinct("department", "salary"))
df2.show(truncate=False)
print("Distinct Count of Department &amp; Salary: "+str(df2.collect()[0][0]))

print("count: "+str(df.select(count("salary")).collect()[0]))
df.select(first("salary")).show(truncate=False)
df.select(last("salary")).show(truncate=False)
df.select(kurtosis("salary")).show(truncate=False)
df.select(max("salary")).show(truncate=False)
df.select(min("salary")).show(truncate=False)
df.select(mean("salary")).show(truncate=False)
df.select(skewness("salary")).show(truncate=False)
df.select(stddev("salary"), stddev_samp("salary"), \
    stddev_pop("salary")).show(truncate=False)
df.select(sum("salary")).show(truncate=False)
df.select(sumDistinct("salary")).show(truncate=False)
df.select(variance("salary"),var_samp("salary"),var_pop("salary")) \
  .show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

